In [ ]:
import pandas as pd
import geopandas as gpd
import subprocess
import numpy as np
import shapely.geometry as shpg
import os
import sys
import json
import logging

In [ ]:
sys.path.append(os.getcwd() + '/../..')
from utils import mkdir, open_zip_shapefile, open_tar_shapefile, haversine

## Files and storage paths

In [ ]:
# Region of interest
reg = 1

# go down from rgi7_scripts/workflow
data_dir = '../../../../rgi7_data/'

# Input dirctory
input_dir_tar = os.path.join(data_dir, 'l4_rgi7b0_tar')
input_dir = os.path.join(data_dir, 'l4_rgi7b0')

In [ ]:
# Parameters
reg = 7


### Load the input data

In [ ]:
reg_file = os.path.join(data_dir, '00_rgi70_regions', '00_rgi70_O1Regions')
reg_file = gpd.read_file(reg_file)
fname = reg_file.loc[reg_file['o1region'] == f'{reg:02d}'].iloc[0]['long_code']

In [ ]:
# Read beta files
shp = open_tar_shapefile(input_dir_tar + f'/RGI2000-v7.0-G-{fname}.tar.gz')
odf_subm = pd.read_csv(input_dir + f'/RGI2000-v7.0-G-{fname}/RGI2000-v7.0-G-{fname}_subm_info.csv', index_col=0)

## Printouts for docs 

In [ ]:
for i, line in odf_subm.iterrows():
    print(f"Submission {i}")
    print(f": **Submitter**: {line['submitters']}."
          f"<br/>**Number of outlines**: {line['n_outlines']}. **Area**: {line['area_km2']:.1f}km². **Release date**: {line['release_dt'][:10]}."
          f"<br/>**Analysts**: {line['analysts']}.")
    print(f"")

## Maps 

In [ ]:
import salem
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
from matplotlib.image import imread
import seaborn as sns

In [ ]:
background_medres = '/home/users/fmaussion/www_fmaussion/misc/rgi7_data/l0_support_data/images/natural-earth-1_large8192px_white.png'
background_higres = '/home/users/fmaussion/www_fmaussion/misc/rgi7_data/l0_support_data/images/natural-earth-1_large_white.png'

In [ ]:
rdf = salem.read_shapefile(os.path.join(data_dir, '00_rgi70_regions', '00_rgi70_O1Regions', '00_rgi70_O1Regions.shp'))
srdf = salem.read_shapefile(os.path.join(data_dir, '00_rgi70_regions', '00_rgi70_O2Regions', '00_rgi70_O2Regions.shp'))

rdf = rdf.loc[rdf['o1region'] == f'{reg:02d}']
srdf = srdf.loc[srdf['o1region'] == f'{reg:02d}']

In [ ]:
title = f'RGI{reg:02d} - {rdf.full_name.iloc[0]}'
title

In [ ]:
extent_shape = [rdf.min_x.iloc[0], rdf.max_x.iloc[0], rdf.min_y.iloc[0], rdf.max_y.iloc[0]]
extent_shape

In [ ]:
reg_str = f'{reg:02d}'

In [ ]:
region_params = {
    '01': {
        'extent': [-180.0, -132, 46.0, 72.0],
        'o1_legend_loc': 'upper left',
        'o2_legend_loc': 'lower left',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 0.2,
        'background': background_medres,
        'figsize': (10, 6),
    },
    '02': {
        'extent': [-136, -102.0, 32.0, 75.0],
        'o1_legend_loc': 'upper left',
        'o2_legend_loc': 'lower left',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 0.4,
        'background': background_medres,
        'figsize': (5, 9),
    },
    '03': {
        'extent': [-123, -76.0, 72.0, 86.0],
        'o1_legend_loc': 'upper left',
        'o2_legend_loc': 'lower left',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 0.2,
        'background': background_medres,
        'figsize': (10, 8),
    },
    '04': {
        'extent': [-90.0, -58.0, 55.0, 75.0],
        'o1_legend_loc': 'upper left',
        'o2_legend_loc': 'lower left',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 0.5,
        'background': background_medres,
        'figsize': (10, 8),
    },
    '05': {
        'extent': [-60.0, -16.0, 58.0, 86.0],
        'o1_legend_loc': 'upper left',
        'o2_legend_loc': 'lower left',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 0.4,
        'background': background_medres,
        'figsize': (10, 8),
    },
    '06': {
        'extent': [-26.5, -9.5, 62, 67.5],
        'o1_legend_loc': 'upper left',
        'o2_legend_loc': 'lower left',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 0.4,
        'background': background_higres,
        'figsize': (10, 8),
    },
    '07': {
        'extent': [-10.5, 35.5, 69.4, 83.2],
        'o1_legend_loc': 'upper left',
        'o2_legend_loc': 'lower left',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 0.2,
        'background': background_higres,
        'figsize': (10, 8),
    },
    '08': {
        'extent': [4.0, 35.0, 57.0, 74.5],
        'o1_legend_loc': 'upper left',
        'o2_legend_loc': 'lower right',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 0.4,
        'background': background_higres,
        'figsize': (10, 8),
    },
    '09': {
        'extent': [34.5, 95.0, 70.0, 84.5],
        'o1_legend_loc': 'upper left',
        'o2_legend_loc': 'lower right',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 0.4,
        'background': background_medres,
        'figsize': (10, 8),
    },
    '10': {
        'extent': [75, 150, 31, 88],
        'o1_legend_loc': 'upper left',
        'o2_legend_loc': 'lower left',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 0.4,
        'background': background_medres,
        'figsize': (10, 8),
    },
    '11': {
        'extent': [-6.0, 26.0, 39.0, 50.2],
        'o1_legend_loc': 'upper left',
        'o2_legend_loc': 'lower left',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 0.9,
        'background': background_higres,
        'figsize': (12, 8),
    },
    '12': {
        'extent': [32.0, 54.0, 29.0, 45.2],
        'o1_legend_loc': 'upper left',
        'o2_legend_loc': 'lower left',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 1.2,
        'background': background_higres,
        'figsize': (10, 8),
    },
    '13': {
        'extent': [65.0, 105.0, 27., 46.2],
        'o1_legend_loc': 'upper left',
        'o2_legend_loc': 'lower left',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 0.8,
        'background': background_higres,
        'figsize': (10, 8),
    },
    '14': {
        'extent': [65.0, 82, 25.2, 38],
        'o1_legend_loc': 'upper left',
        'o2_legend_loc': 'lower right',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 0.4,
        'background': background_higres,
        'figsize': (10, 8),
    },
    '15': {
        'extent': [75.4, 105.0, 25, 32.2],
        'o1_legend_loc': 'lower left',
        'o2_legend_loc': 'lower right',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 0.4,
        'background': background_higres,
        'figsize': (10, 8),
    },
    '16': {
        'extent': [-101.0, 143.0, -26.0, 21.0],
        'o1_legend_loc': (0.2, 0.05),
        'o2_legend_loc': (0.6, 0.05),
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 1.0,
        'background': background_medres,
        'figsize': (12, 8),
    },
    '17': {
        'extent': [-83.0, -61.0, -58.0, -24.0],
        'o1_legend_loc': 'upper left',
        'o2_legend_loc': 'lower left',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 0.4,
        'background': background_medres,
        'figsize': (10, 8),
    },
    '18': {
        'extent': [164.0, 179.0, -49.5, -33.5],
        'o1_legend_loc': 'upper left',
        'o2_legend_loc': 'lower left',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 0.8,
        'background': background_higres,
        'figsize': (10, 8),
    },
    '19': {
        'extent': [-180, 180, -90, -47],
        'o1_legend_loc': 'upper left',
        'o2_legend_loc': 'lower left',
        'o1_legend_loc_subm': 'upper right',
        'linewidth_o1': 1.5,
        'linewidth_o2': 1,
        'linewidth_outlines': 0.9,
        'background': background_medres,
        'figsize': (10, 8),
    },
}

region_params = region_params[reg_str]

In [ ]:
extent = region_params['extent']

In [ ]:
region_proj = {
    '01': ccrs.LambertConformal(central_longitude=(extent[0] + extent[1]) / 2, 
                                central_latitude=(extent[2] + extent[3]) / 2, 
                                standard_parallels=(50, 65)),
    '02': ccrs.LambertConformal(central_longitude=(extent[0] + extent[1]) / 2, 
                                central_latitude=(extent[2] + extent[3]) / 2, 
                                standard_parallels=(40, 60)),
    '03': ccrs.LambertConformal(central_longitude=(extent[0] + extent[1]) / 2, 
                                central_latitude=(extent[2] + extent[3]) / 2, 
                                standard_parallels=(76, 82)),
    '04': ccrs.LambertConformal(central_longitude=(extent[0] + extent[1]) / 2, 
                                central_latitude=(extent[2] + extent[3]) / 2, 
                                standard_parallels=(62, 70)),
    '05': ccrs.LambertConformal(central_longitude=(extent[0] + extent[1]) / 2, 
                                central_latitude=(extent[2] + extent[3]) / 2, 
                                standard_parallels=(62, 70)),
    '06': ccrs.LambertConformal(central_longitude=(extent[0] + extent[1]) / 2, 
                                central_latitude=(extent[2] + extent[3]) / 2, 
                                standard_parallels=(64, 66)),
    '07': ccrs.LambertConformal(central_longitude=(extent[0] + extent[1]) / 2, 
                                central_latitude=(extent[2] + extent[3]) / 2, 
                                standard_parallels=(72, 80)),
    '08': ccrs.LambertConformal(central_longitude=(extent[0] + extent[1]) / 2, 
                                central_latitude=(extent[2] + extent[3]) / 2, 
                                standard_parallels=(40, 60)),
    '09': ccrs.LambertConformal(central_longitude=(extent[0] + extent[1]) / 2, 
                                central_latitude=(extent[2] + extent[3]) / 2, 
                                standard_parallels=(75, 80)),
    '10': ccrs.LambertConformal(central_longitude=(extent[0] + extent[1]) / 2, 
                                central_latitude=(extent[2] + extent[3]) / 2, 
                                standard_parallels=(60, 80)),
    '11': ccrs.LambertConformal(central_longitude=(extent[0] + extent[1]) / 2,
                                central_latitude=(extent[2] + extent[3]) / 2,
                                standard_parallels=(42, 48)),
    '12': ccrs.LambertConformal(central_longitude=(extent[0] + extent[1]) / 2,
                                central_latitude=(extent[2] + extent[3]) / 2,
                                standard_parallels=(34, 50)),
    '13': ccrs.LambertConformal(central_longitude=(extent[0] + extent[1]) / 2,
                                central_latitude=(extent[2] + extent[3]) / 2,
                                standard_parallels=(34, 41)),
    '14': ccrs.LambertConformal(central_longitude=(extent[0] + extent[1]) / 2,
                                central_latitude=(extent[2] + extent[3]) / 2,
                                standard_parallels=(30, 36)),
    '15': ccrs.LambertConformal(central_longitude=(extent[0] + extent[1]) / 2,
                                central_latitude=(extent[2] + extent[3]) / 2,
                                standard_parallels=(27, 30)),
    '16': ccrs.Mercator(central_longitude=(extent[0] + extent[1]) / 2),
    '17': ccrs.Mercator(central_longitude=(extent[0] + extent[1]) / 2),
    '18': ccrs.LambertConformal(central_longitude=(extent[0] + extent[1]) / 2,
                                central_latitude=(extent[2] + extent[3]) / 2,
                                standard_parallels=(-38, -44)),
    '19': ccrs.SouthPolarStereo(),
}

In [ ]:
map_crs = region_proj[reg_str]

In [ ]:
shp_repr = shp.to_crs(map_crs)

In [ ]:
o2_cmap = sns.color_palette('husl', n_colors=len(srdf))
subm_cmap = sns.color_palette('bright')
color_order = [4, 5, 6, 8, 9, 2, 1, 0, 3, 7]

for is_rgi6 in [True, False]:
    
    print(f'Plot {is_rgi6}')
    
    # Prepare figure
    fig = plt.figure(figsize=region_params['figsize'])
    ax = fig.add_subplot(1, 1, 1, projection=map_crs)
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    
    if reg_str == '19':
        import matplotlib.path as mpath
        theta = np.linspace(0, 2*np.pi, 100)
        map_circle = mpath.Path(np.vstack([np.sin(theta), np.cos(theta)]).T * 0.5 + [0.5, 0.5])
        ax.set_boundary(map_circle, transform=ax.transAxes)
        gl = ax.gridlines(draw_labels=True, x_inline=False, y_inline=False, dms=True,  
                          xlocs=np.arange(9) * 45 - 180);
    else:
        gl = ax.gridlines(draw_labels=True, x_inline=False, y_inline=False, dms=True, 
                          xlocs=np.arange(37) * 10 - 180);
        gl.top_labels = False
        gl.right_labels = False
    
    # Add background
    ax.imshow(imread(region_params['background']), 
              origin='upper', 
              transform=ccrs.PlateCarree(), 
              extent=[-180, 180, -90, 90])
    
    # Plot o1 regions
    if reg_str != '19':
        rdf.to_crs(map_crs).plot(ax=ax, edgecolor='k', facecolor='none', zorder=4, 
                                 linewidth=region_params['linewidth_o1'])
    
    # Plot o2 regions including legend 
    sr_handles = []
    for ind, row in srdf.reset_index().to_crs(map_crs).iterrows():
        ec = o2_cmap[ind]
        s_loc = srdf.to_crs(map_crs).loc[srdf['o2region'] == row['o2region']]
        s_loc.plot(ax=ax, facecolor='none', edgecolor=ec, zorder=5, linewidth=region_params['linewidth_o2'], ls='--')
        sr_handles.append(mpatches.Patch(facecolor='none', edgecolor=ec,
                                         linestyle='--', 
                                         linewidth=region_params['linewidth_o2'], 
                                         label=f'{row.o2region}'))

    sr_leg = ax.legend(handles=sr_handles, loc=region_params['o2_legend_loc'], title='O2 Regions')
    art = ax.add_artist(sr_leg)
    art.zorder = 99
    
    handles = []
    if is_rgi6:
        for itis in [True, False]:
            ec = '#001fff' if itis else '#ff0000'
            fc = '#465dff' if itis else '#ff4646'
            label = 'Same as RGI6' if itis else 'New in RGI7'
            s_loc = shp_repr.loc[shp_repr['is_rgi6'] == itis]
            if len(s_loc) > 0:
                s_loc.plot(ax=ax, facecolor=fc, edgecolor=ec, 
                           linewidth=region_params['linewidth_outlines'], zorder=97)
                area = s_loc['area_km2'].sum()
            else:
                area = 0
            handles.append(mpatches.Patch(facecolor=ec, label=f'{label}'))
    else:
        for i, subid in enumerate(sorted(shp.subm_id.unique().astype(int))):
            fc = subm_cmap[color_order[i]]
            ec = fc # subm_cmap_ec[i]
            s_loc = shp_repr.loc[shp.subm_id == subid]
            s_loc.plot(ax=ax, facecolor=fc, edgecolor=ec, linewidth=region_params['linewidth_outlines'], zorder=97)
            area = s_loc['area_km2'].sum()
            handles.append(mpatches.Patch(facecolor=fc, label=f'{subid}'))

    # Alternative solution for "Polygon handles not passing to legend"
    try: 
        loc = region_params['o1_legend_loc_subm']
    except KeyError:
        loc = region_params['o1_legend_loc']
    ax.legend(handles=handles, loc=loc);
    ax.set_title(title);

    plot_dir = data_dir + f'/l4_rgi7b0_plots/RGI{reg:02d}'
    mkdir(plot_dir)
    plotname = 'isrgi6_map' if is_rgi6 else 'inventory_map'
    plt.savefig(plot_dir + f'/{plotname}.jpeg', bbox_inches='tight', dpi=300)
    # plt.savefig(f'RGI{reg_str}_{plotname}.jpeg', bbox_inches='tight', dpi=300)

### Histogram 

In [ ]:
figsize=(8, 7)

f, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize)

date_i = shp['src_date'].str[:4].astype(int).to_frame('src_date')
date_i['area'] = shp['area_km2']
date = date_i.groupby('src_date')['area'].sum().to_frame()
date['area'] = date['area'] / date['area'].sum()
date['count'] =  date_i.groupby('src_date')['area'].count()
date = date.reset_index()

sns.barplot(ax=ax1, x='src_date', y='area', data=date, color='lightsteelblue');
ax1.set_ylabel('Relative area'); ax1.set_xlabel(''); 
ax1.set_title(title);
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)

sns.barplot(ax=ax2, x='src_date', y='count', data=date, color='thistle');
ax2.set_ylabel('Number of outlines'); ax2.set_xlabel('Source date'); 
plt.xticks(rotation=45, ha='center')

plot_dir = data_dir + f'/l4_rgi7b0_plots/RGI{reg:02d}'
mkdir(plot_dir)
plotname = 'date_hist'
plt.tight_layout()
plt.savefig(plot_dir + f'/{plotname}.png', bbox_inches='tight', dpi=300)